## 基于 Embedding 与 CNN 进行文本分类

In [ ]:
import tensorflow.keras as keras
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
% matplotlib inline
import numpy as np
import sklearn
import os
import sys
import time
import tensorflow as tf

In [ ]:
print(tf.__version__)
print("GPU : ", tf.test.is_gpu_available())
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

### 1. 数据集读取与词表构建

#### 1.1 读取数据

In [ ]:
# imdb 是电影评论数据集, 类别为积极与消极
imdb = keras.datasets.imdb
# 词表数量, 前 10000 个会被保存, 其他的会被当做特殊字符处理
vocab_size = 10000
# 词表的index从index_from 开始算, 因为要设置特殊字符! 0 ~ 3 是特殊字符
index_from = 3

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=vocab_size, index_from=index_from)

In [ ]:
# 打印向量
print(train_data[0], train_labels[0])
print(train_data.shape, train_labels.shape)
print(len(train_data[0]), len(train_data[1]))

In [ ]:
print(test_data.shape, test_labels.shape)

#### 1.2 构建词表

In [ ]:
# 获取词表， word_index = (单词, 索引)
word_index = imdb.get_word_index()
print(len(word_index))

In [ ]:
print(type(word_index))
# 打印前10个词表
for word, index in word_index.items():
    if int(index) < 30:
        print(word, index)

In [ ]:
# 因为之前的设置 index_from = 3, 所以所有的索引都需要偏移3
word_index = {word: (index + 3) for word, index in word_index.items()}

In [ ]:
# 设置特殊字符
# <PAD> padding 的填充字符
# <START> 句子的开头的填充字符
# <UNK> 找不到字符时, 返回UNK
# <END> 句子的末尾的特殊字符

word_index['<PAD>'] = 0
word_index['<START>'] = 1
word_index['<UNK>'] = 2
word_index['<END>'] = 3
reverse_word_index = dict([(index, word) for word, index in word_index.items()])

In [ ]:
# 使用词表反向解析句子
def decode_review(text_ids):
    return " ".join(
        [reverse_word_index.get(word_id, "<UNK>") for word_id in text_ids]
    )


# 解析数字为真实文本
decode_review(train_data[0])

### 2. 数据padding

- padding 就是对数据进行变长处理
    1. 处理方式: 如果数据过长, 就截断数据, 如果数据过短就使用字符(0)填充
    2. 合并
- keras.preprocessing.sequence.pad_sequences
    1. value=word_index['<PAD>'] , 使用词表中 'PAD' 对应的部分进行填充
    2. padding='post' , value : post or pre , post 就是 把 padding 放到句子的后面, pre 就是 把padding 放到句子的前面
    3. maxlen=max_length 句子的最大长度

In [ ]:
# 设置的单个句子的最长的长度

max_length = 500

# 对训练数据进行padding

train_data = keras.preprocessing.sequence.pad_sequences(
    train_data,  # integer of list
    value=word_index['<PAD>'],  # 使用词表中 'PAD' 对应的部分进行填充
    padding='post',  # value : post or pre , post 就是 把 padding 放到句子的后面, pre 就是 把padding 放到句子的前面
    maxlen=max_length  # 句子的最大长度
)

test_data = keras.preprocessing.sequence.pad_sequences(
    test_data,  # integer of list
    value=word_index['<PAD>'],  # 使用词表中 'PAD' 对应的部分进行填充
    padding='post',  # value : post or pre , post 就是 把 padding 放到句子的后面, pre 就是 把padding 放到句子的前面
    maxlen=max_length  # 句子的最大长度
)

# 打印样本
print(train_data.shape)

### 3. 模型构建

用一个例子解释textCNN的设计 :
1. 这里的输入是一个有11个词的句子，每个词用6维词向量表示。因此输入序列的宽为11，输入通道数为6。
2. 给定2个一维卷积核，核宽分别为2和4，输出通道数分别设为4和5。
3. 因此，一维卷积计算后，4个输出通道的宽为11−2+1=10, 而其他5个通道的宽为11−4+1=8。
4. 尽管每个通道的宽不同，我们依然可以对各个通道做时序最大池化，并将9个通道的池化输出连结成一个9维向量。最终，使用全连接将9维向量变换为2维输出，即正面情感和负面情感的预测



In [ ]:
# 每个word都转换成长度为 16 的向量
# 每个词用长度为 embedding_dim 的向量表示
embedding_dim = 16
# 每个单词的最大长度(文本序列长度)
sentence_max_length = max_length
# 输入样本的宽为sentence_max_length, 高为1, 输入通道数为embedding_dim

batch_size = 128

# 定义模型
text_cnn_model = keras.models.Sequential([
    # 1. 定义一个 [vocab_size, embedding_dim] 的矩阵
    # 2. 注意 : train_data : [1, 14, 22, 16, 43, 530, 973, ... ] 类似于这样的索引, 1 . 14 这种的是词表中对应单词的索引
    # 3. embedding 对每一个单词生成一个长度为 embedding_dim(16) 的向量(是向量不是矩阵, 是一维的, 注意是每一个单词)
    # 4. 一个句子 train_data[i] 会生成一个 embedding_dim * max_length 的二位矩阵 [max_length, embedding_dim]
    # 5. 最终会生成一个 batch_size * max_length * embedding_dim 的 三维矩阵
    keras.layers.Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        input_length=max_length
    ),
    # 添加一维卷积层
    # filters : 卷积核的数量
    # kernel_size : 卷积核的大小
    # padding : 要不要给输入输出加上padding, 让输入和输出大小一样
    keras.layers.Conv1D(filters= 64, kernel_size=3, padding="valid", strides=1, activation='relu'),
    keras.layers.MaxPool1D(),
    keras.layers.Conv1D(filters= 64, kernel_size=4, padding="valid", strides=1, activation='relu'),
    keras.layers.MaxPool1D(),
    keras.layers.Conv1D(filters= 64, kernel_size=5, padding="valid", strides=1, activation='relu'),
    keras.layers.MaxPool1D(),
    keras.layers.Flatten(),
    keras.layers.Dense(units=128),
    # dropout
    keras.layers.Dropout(rate=0.2),
    keras.layers.ReLU(),
    # 输出分类结果
    keras.layers.Dense(2, activation="softmax")
])

text_cnn_model.summary()

- Adam Optimizer是对SGD的扩展，可以代替经典的随机梯度下降法来更有效地更新网络权重
    1. 计算效率高
    2. 很少的内存需求
    3. 梯度的对角线重缩放不变（这意味着亚当将梯度乘以仅带正因子的对角矩阵是不变的，以便更好地理解此堆栈交换）
    4. 非常适合数据和/或参数较大的问题
    5. 适用于非固定目标
    6. 适用于非常嘈杂和/或稀疏梯度的问题
    7. 超参数具有直观的解释，通常需要很少的调整（我们将在配置部分中对此进行详细介绍）

- binary_crossentropy : 二进制交叉熵
    1. 参考 : https://blog.csdn.net/qq_35599937/article/details/105608354

In [ ]:
text_cnn_model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [ ]:
callbacks = [
    keras.callbacks.EarlyStopping(patience=3, min_delta=1e-3)
]
history = text_cnn_model.fit(
    train_data, train_labels, epochs=30, batch_size=batch_size,
    validation_split=0.2,  # 设置验证集为 20%
    #callbacks=callbacks
)

In [ ]:
history.history

### 4. 绘制图像

In [ ]:
def plot_learning_curves(history, label, epochs, min_value, max_value):
    data = {}
    data[label] = history.history[label]
    data["val_" + label] = history.history["val_" + label]
    pd.DataFrame(data).plot(figsize=(8, 5))
    plt.grid(True)
    plt.axis([0, epochs, min_value, max_value])
    plt.show()


# val_accuracy : 验证集的准确率
# val_loss : 验证集的误差
# 如下图可以看到, 验证集上的loss一段平稳过后直线上升, 说明出现了过拟合
plot_learning_curves(history, 'accuracy', 30, 0, 1)
plot_learning_curves(history, 'loss', 30, 0, 3)

In [ ]:
# 使用测试集进行验证
text_cnn_model.evaluate(test_data, test_labels, batch_size=batch_size)